In [1]:
import os
import csv
from collections import defaultdict

def create_tracklets(annotation_file, max_gap=3):
    """
    Creates tracklets from the given annotation file by grouping consecutive detections of the same object.

    Args:
    - annotation_file (str): Path to the annotation text file.
    - max_gap (int): Maximum gap allowed before creating a new tracklet.

    Returns:
    - tracklets (dict): Dictionary where keys are tracklet IDs and values are lists of (frame_id, object_id, x, y, width, height, score, class, visibility).
    """

    object_tracks = defaultdict(list)  # Stores all detections per object_id
    tracklets = {}  # Stores fragmented tracklets
    tracklet_id = 0  # Unique tracklet identifier

    # Read annotation file
    with open(annotation_file, "r") as f:
        reader = csv.reader(f)
        for row in reader:
            try:
                frame_id, object_id, x, y, width, height, score, obj_class, visibility, ignored = map(float, row)
                object_id = int(object_id)
                frame_id = int(frame_id)

                object_tracks[object_id].append((frame_id, object_id, x, y, width, height, score, obj_class, visibility))

            except ValueError:
                print(f"Skipping malformed line: {row}")

    # Create tracklets by detecting gaps in object tracking
    for obj_id, detections in object_tracks.items():
        detections.sort()  # Ensure detections are sorted by frame_id

        last_frame = -1
        current_tracklet = []

        for detection in detections:
            frame_id = detection[0]

            if last_frame != -1 and (frame_id - last_frame) > max_gap:
                # Start a new tracklet if gap is too large
                tracklets[tracklet_id] = current_tracklet
                tracklet_id += 1
                current_tracklet = []

            current_tracklet.append(detection)
            last_frame = frame_id

        # Store remaining tracklet
        if current_tracklet:
            tracklets[tracklet_id] = current_tracklet
            tracklet_id += 1

    return tracklets

def process_all_files(annotation_folder, output_folder, max_gap=3):
    """
    Processes all annotation files in a folder and saves tracklets for each file.

    Args:
    - annotation_folder (str): Folder containing annotation files.
    - output_folder (str): Folder to save tracklet files.
    - max_gap (int): Maximum gap allowed before creating a new tracklet.
    """

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file in os.listdir(annotation_folder):
        if file.endswith(".txt"):  # Process only text files
            annotation_file = os.path.join(annotation_folder, file)
            tracklets = create_tracklets(annotation_file, max_gap)

            output_file = os.path.join(output_folder, file.replace(".txt", "-tracklets.txt"))

            # Save tracklets
            with open(output_file, "w", newline="") as f:
                writer = csv.writer(f)
                for tracklet_id, frames in tracklets.items():
                    for frame in frames:
                        writer.writerow([tracklet_id] + list(frame))

            print(f"Saved tracklets for {file} -> {output_file}")

# Example Usage
annotation_folder = r"F:\edited visdrone\annotations"
output_folder = r"F:\edited visdrone\tracklets"
process_all_files(annotation_folder, output_folder)


Saved tracklets for uav0000289_00001_v-1.txt -> F:\edited visdrone\tracklets\uav0000289_00001_v-1-tracklets.txt
Saved tracklets for uav0000308_00000_v-1.txt -> F:\edited visdrone\tracklets\uav0000308_00000_v-1-tracklets.txt
Saved tracklets for uav0000323_01173_v-1.txt -> F:\edited visdrone\tracklets\uav0000323_01173_v-1-tracklets.txt
Saved tracklets for uav0000326_01035_v-1.txt -> F:\edited visdrone\tracklets\uav0000326_01035_v-1-tracklets.txt
Saved tracklets for uav0000329_04715_v-1.txt -> F:\edited visdrone\tracklets\uav0000329_04715_v-1-tracklets.txt


In [6]:
import os
import glob
import numpy as np
from collections import defaultdict

def compute_tracklet_metrics(tracklets_folder, output_folder):
    tracklet_files = glob.glob(os.path.join(tracklets_folder, "*.txt"))

    if not tracklet_files:
        print("No tracklet files found in the folder.")
        return

    os.makedirs(output_folder, exist_ok=True)  # Ensure output folder exists

    for file_path in tracklet_files:
        with open(file_path, "r") as f:
            lines = f.readlines()

        if not lines:
            print(f"Warning: {file_path} is empty.")
            continue

        tracklet_data = defaultdict(list)

        for line in lines:
            parts = line.strip().split(",")
            if len(parts) < 10:
                continue  # Skip malformed lines

            tracklet_id = int(parts[0])  # Unique tracklet ID
            frame_id = int(parts[1])     # Frame number
            x, y = float(parts[3]), float(parts[4])  # Ignore parts[2], use parts[3] and parts[4]
            width, height = float(parts[5]), float(parts[6])

            tracklet_data[tracklet_id].append((frame_id, x, y, width, height))

        output_file_path = os.path.join(output_folder, os.path.basename(file_path))

        with open(output_file_path, "w") as f_out:
            for tracklet_id, data in tracklet_data.items():
                if len(data) < 2:
                    print(f"Skipping Tracklet {tracklet_id} in {file_path} (not enough frames to compute velocity).")
                    continue

                data.sort()  # Sort by frame_id

                velocities = []
                areas = []
                aspect_ratios = []

                for i in range(1, len(data)):
                    prev_frame, prev_x, prev_y, _, _ = data[i - 1]
                    curr_frame, curr_x, curr_y, width, height = data[i]

                    # Compute velocity (Euclidean distance per frame difference)
                    frame_diff = curr_frame - prev_frame
                    if frame_diff > 0:
                        velocity = np.sqrt((curr_x - prev_x) ** 2 + (curr_y - prev_y) ** 2) / frame_diff
                        velocities.append(velocity)

                    # Compute bounding box area
                    area = width * height
                    areas.append(area)

                    # Compute aspect ratio
                    aspect_ratio = width / height if height > 0 else 0
                    aspect_ratios.append(aspect_ratio)

                avg_velocity = np.mean(velocities) if velocities else 0
                avg_area = np.mean(areas) if areas else 0
                avg_aspect_ratio = np.mean(aspect_ratios) if aspect_ratios else 0

                # Write to file in CSV format: Tracklet ID, Avg Velocity, Avg Area, Avg Aspect Ratio
                f_out.write(f"{tracklet_id},{avg_velocity:.3f},{avg_area:.3f},{avg_aspect_ratio:.3f}\n")

        print(f"Metrics saved to {output_file_path}")

# Example usage
tracklets_folder = r"F:\edited visdrone\tracklets"  # Change this to your input folder path
output_folder = r"F:\edited visdrone\processed_tracklets"  # Change this to your output folder path
compute_tracklet_metrics(tracklets_folder, output_folder)


Metrics saved to F:\edited visdrone\processed_tracklets\uav0000289_00001_v-1-tracklets.txt
Metrics saved to F:\edited visdrone\processed_tracklets\uav0000308_00000_v-1-tracklets.txt
Metrics saved to F:\edited visdrone\processed_tracklets\uav0000323_01173_v-1-tracklets.txt
Metrics saved to F:\edited visdrone\processed_tracklets\uav0000326_01035_v-1-tracklets.txt
Metrics saved to F:\edited visdrone\processed_tracklets\uav0000329_04715_v-1-tracklets.txt


In [9]:
import os
import glob
import cv2
import numpy as np
from collections import defaultdict

def compute_color_histogram(image, bbox):
    """
    Computes the average color histogram for a given bounding box in an image.

    Args:
    - image (np.array): Input image.
    - bbox (tuple): (x, y, width, height) of the bounding box.

    Returns:
    - hist (np.array): Flattened color histogram (normalized).
    """
    x, y, w, h = map(int, bbox)
    roi = image[y:y+h, x:x+w]  # Crop the bounding box

    if roi.size == 0:
        return np.zeros((512,))  # Return zero histogram if region is invalid

    hist = cv2.calcHist([roi], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()  # Normalize and flatten
    return hist

def compute_sift_descriptors(image, bbox):
    """
    Computes the average SIFT descriptor for a given bounding box in an image.

    Args:
    - image (np.array): Input image.
    - bbox (tuple): (x, y, width, height) of the bounding box.

    Returns:
    - avg_descriptor (np.array): Mean SIFT descriptor (128D) or zero vector if none found.
    """
    x, y, w, h = map(int, bbox)
    roi = image[y:y+h, x:x+w]  # Crop the bounding box

    if roi.size == 0:
        return np.zeros((128,))  # Return zero descriptor if region is invalid

    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(roi, None)

    if descriptors is None:
        return np.zeros((128,))  # Return zero descriptor if no keypoints found

    return np.mean(descriptors, axis=0)  # Compute average descriptor

def process_tracklet_features(tracklets_folder, frames_folder, output_folder):
    """
    Computes the average color histogram and SIFT descriptor for each tracklet.

    Args:
    - tracklets_folder (str): Folder containing tracklet annotation files.
    - frames_folder (str): Folder containing image sequences.
    - output_folder (str): Folder to save the extracted features.
    """
    tracklet_files = glob.glob(os.path.join(tracklets_folder, "*.txt"))

    if not tracklet_files:
        print("No tracklet files found in the folder.")
        return

    os.makedirs(output_folder, exist_ok=True)  # Ensure output folder exists

    for file_path in tracklet_files:
        tracklet_data = defaultdict(list)

        with open(file_path, "r") as f:
            lines = f.readlines()

        if not lines:
            print(f"Warning: {file_path} is empty.")
            continue

        for line in lines:
            parts = line.strip().split(",")
            if len(parts) < 10:
                continue  # Skip malformed lines

            tracklet_id = int(parts[0])  # Unique tracklet ID
            frame_id = int(parts[1])     # Frame number
            x, y, width, height = map(float, parts[3:7])

            tracklet_data[tracklet_id].append((frame_id, x, y, width, height))

        # Determine the sequence folder from filename (assuming folder names match video names)
        video_name = os.path.basename(file_path).replace("-1-tracklets.txt", "")
        sequence_folder = os.path.join(frames_folder, video_name)

        if not os.path.exists(sequence_folder):
            print(f"Warning: Missing sequence folder {sequence_folder}. Skipping file {file_path}.")
            continue

        output_file_path = os.path.join(output_folder, os.path.basename(file_path))

        with open(output_file_path, "w") as f_out:
            for tracklet_id, bboxes in tracklet_data.items():
                color_histograms = []
                sift_descriptors = []

                for frame_id, x, y, w, h in bboxes:
                    image_path = os.path.join(sequence_folder, f"{frame_id:07d}.jpg")  # Assumes 7-digit frame filenames

                    if not os.path.exists(image_path):
                        continue  # Skip missing frames

                    image = cv2.imread(image_path)
                    if image is None:
                        continue  # Skip corrupted images

                    hist = compute_color_histogram(image, (x, y, w, h))
                    sift_desc = compute_sift_descriptors(image, (x, y, w, h))

                    color_histograms.append(hist)
                    sift_descriptors.append(sift_desc)

                if not color_histograms or not sift_descriptors:
                    print(f"Skipping Tracklet {tracklet_id} in {file_path} (no valid images).")
                    continue

                avg_histogram = np.mean(color_histograms, axis=0)
                avg_sift_descriptor = np.mean(sift_descriptors, axis=0)

                # Save as CSV format: tracklet_id, histogram values, sift descriptor values
                hist_str = ",".join(map(str, avg_histogram))
                sift_str = ",".join(map(str, avg_sift_descriptor))

                f_out.write(f"{tracklet_id},{hist_str},{sift_str}\n")

        print(f"Features saved to {output_file_path}")

# Example Usage
tracklets_folder = r"F:\edited visdrone\tracklets"  # Change this to your tracklet folder
frames_folder = r"F:\edited visdrone\sequences"  # Change this to your image sequence folder
output_folder = r"F:\edited visdrone\sift_and_histogram"  # Change this to your output folder
process_tracklet_features(tracklets_folder, frames_folder, output_folder)


Features saved to F:\edited visdrone\sift_and_histogram\uav0000289_00001_v-1-tracklets.txt
Features saved to F:\edited visdrone\sift_and_histogram\uav0000308_00000_v-1-tracklets.txt
Features saved to F:\edited visdrone\sift_and_histogram\uav0000323_01173_v-1-tracklets.txt
Features saved to F:\edited visdrone\sift_and_histogram\uav0000326_01035_v-1-tracklets.txt
Features saved to F:\edited visdrone\sift_and_histogram\uav0000329_04715_v-1-tracklets.txt
